# 2019 novel coronavirus data extraction from text

Currently only support extraction from the official daily briefing of government 
of the Hubei province, whose capital is Wuhan. 

This repo is dedicated to all who died and helped in this tragedy, especially Dr. [Li Wenliang](https://en.wikipedia.org/wiki/Li_Wenliang). 

In [117]:
s1="2020年1月28日0时-24时，湖北省新增新型冠状病毒感染的肺炎病例840例（武汉市新增315例、黄石市新增33例、十堰市新增23例、襄阳市新增61例、宜昌市新增12例、荆州市新增30例、荆门市新增28例、鄂州新增27例、孝感市新增101例、黄冈市新增111例、咸宁市新增21例、随州市新增46例、恩施州新增13例、仙桃市新增5例、天门市新增11例，潜江新增1例，神农架林区新增2例）。全省新增死亡25例，其中：武汉市19例，孝感市2例，荆门、鄂州、黄冈、天门各1例"

s2="截至2020年1月28日24时，湖北省累计报告新型冠状病毒感染的肺炎病例3554例（其中武汉市1905例、黄石市86例、十堰市88例、襄阳市131例、宜昌市63例、荆州市101例、荆门市142例、鄂州市84例、孝感市274例、黄冈市324例、咸宁市112例、随州市116例、恩施州51例、仙桃市32例、天门市34例、潜江8例，神农架林区3例），已治愈出院80例，死亡125例(其中武汉市104例,黄石市1例、宜昌市1例、荆州市2例、荆门市4例、鄂州市1例，孝感3例、黄冈市5例、天门3例，潜江1例)。目前仍在院治疗3349例，其中重症671例、危重症228例，均在定点医疗机构接受隔离治疗。累计追踪密切接触者22095人，尚在接受医学观察20366人。"

s3= "1月27日，全省发热门诊共接诊31639人，留观3644人。武汉市10702人，留观600; 黄石市1186人，留观90人; 十堰市2136人，留观300人;襄阳市2065人，留观209人; 宜昌市1086人，留观127人; 荆州市1817人，留观231人; 荆门市935人，留观116人; 鄂州市609人，留观229人; 孝感市2812人，留观535人;黄冈市2416人，留观303人;咸宁市1263人，留观92人; 随州市488人，留观214人;恩施州1849人，留观146人;仙桃市1052人，留观244人; 天门市658人，留观143人;潜江市543人，留观65人;神农架林区22人,无留观。"

In [126]:
def triage(s):
    """Separate different categories of numbers
    """

    msg = {}

    segs  = s.split("。")
    # print (segs)

    for seg in segs:
        if "新增" and "病例" in seg:
            msg["increase_patient"] = seg 
        elif "新增死亡" in seg:
            msg["increase_death"] = seg 

    return msg
 
msg = triage(s1)



In [183]:
def parse_increase_patient(s):
    """parse patients increased, return a dict, {location:number}
    """
    import re

    count  = {}

    time_pattern = "2020年(\d{,2})月(\d{,2})日(\d{,2})时-(\d{,2})时"

    patterns = [r'([\u4e00-\u9fa5]*)省新增[\u4e00-\u9fa5]*(\d+)例', 
                r'、([\u4e00-\u9fa5]{1}州)(?!市)新增(\d+)例', # 鄂州新增27例
                r'([\u4e00-\u9fa5]{2,})州(?!市)新增(\d+)例', # 恩施州新增13例 
                r'([\u4e00-\u9fa5]+)市(?<!州)新增(\d+)例',   # 随州市新增46例
                r'([\u4e00-\u9fa5]+)林区新增(\d+)例',   #林區
                r'([\u4e00-\u9fa5]+)(?<![州|市|区])新增(\d+)例',   #others
    ]

    for pattern in patterns: 
        # print (re.findall(pattern, '湖北省新增新型冠状病毒感染的肺炎病例840例（荆门市新增28例、鄂州新增27例、孝感市新增101例、随州市新增46例、恩施州新增13例、天门市新增11例'))
        # print (re.findall(pattern, s))
        for (loc, number) in re.findall(pattern, s):
            count[loc] = int(number)
    return count 

parse_increase_patient(msg["increase_patient"])

{'仙桃': 5,
 '十堰': 23,
 '咸宁': 21,
 '天门': 11,
 '孝感': 101,
 '宜昌': 12,
 '恩施': 13,
 '武汉': 315,
 '湖北': 840,
 '潜江': 1,
 '神农架': 2,
 '荆州': 30,
 '荆门': 28,
 '襄阳': 61,
 '鄂州': 27,
 '随州': 46,
 '黄冈': 111,
 '黄石': 33}

In [185]:
def parse_increase_death(s):
    """parse patients increased, return a dict, {location:number}

    全省新增死亡25例，其中：武汉市19例，孝感市2例，荆门、鄂州、黄冈、天门各1例
    """

    total_patterns = [r'([\u4e00-\u9fa5、，]+)([\d]+)例']
    city_patterns  = [r'([\u4e00-\u9fa5]*)省', 
                r'、([\u4e00-\u9fa5]{1}州)(?!市)', # 鄂州
                r'([\u4e00-\u9fa5]{2,})州(?!市)', # 恩施州
                r'([\u4e00-\u9fa5]+)市(?<!州)',   # 随州市 and all other city cases
                r'([\u4e00-\u9fa5]+)林区',   #林區
    ]

    short_city_pattern = r'([\u4e00-\u9fa5]*)[省|市|州|林区]*$'


    count  = {}

    for pattern in total_patterns: 
        # print (re.findall(pattern, "2020年1月28日0时-24时"))
        for (loc, num) in re.findall(pattern, s):
            print (loc, end="-->")
            if "各" in loc: 
                loc =  loc.replace("各","")
                loc_list = loc.split("、")
                for city_long in loc_list:
                    result = re.match(short_city_pattern, city_long)
                    if result != None:
                        count[result[0]] = int(num)
                    # print (result)
            else:
                for city_pattern in city_patterns: 
                    result = re.findall(city_pattern, loc)
                    if len(result) >0:
                        count[result[0]] = int(num)
                        # print (result[0])
    return count 

parse_increase_death(msg["increase_death"])

全省新增死亡-->武汉市-->，孝感市-->，荆门、鄂州、黄冈、天门各-->

{'全': 25, '天门': 1, '孝感': 2, '武汉': 19, '鄂州': 1, '黄冈': 1}